## Multioutput model

Now, we need a model to detect the type of hate

In [1]:
%load_ext autoreload
%autoreload 2
import json

with open("../data/train.json") as f:
    train_articles = json.load(f)

with open("../data/test.json") as f:
    test_articles = json.load(f)

Let's take just the comments that are HATEFUL

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd

train_comments = [c for article in train_articles for c in article["comments"] if c["is_hateful"]]
test_comments = [c for article in test_articles for c in article["comments"] if c["is_hateful"]]

train_df = pd.DataFrame(train_comments)
test_df = pd.DataFrame(test_comments)

train_df, dev_df = train_test_split(train_df, test_size=0.2)

print(f"We have {len(train_df)} hateful comments in train")
print(f"We have {len(dev_df)} hateful comments in dev")
print(f"We have {len(test_df)} hateful comments in test")

We have 5140 hateful comments in train
We have 1285 hateful comments in dev
We have 1676 hateful comments in test


In [3]:
categories = [
    "calls",
    "WOMEN",
    "LGBTI",
    "RACISM",
    "CLASS",
    "POLITICS",
    "DISABLED",
    "APPEARANCE",
    "CRIMINAL",
]

train_df[categories].mean() - test_df[categories].mean()

calls         0.041419
WOMEN        -0.045314
LGBTI        -0.019344
RACISM        0.036614
CLASS        -0.020343
POLITICS     -0.027052
DISABLED     -0.012728
APPEARANCE   -0.012631
CRIMINAL      0.077931
dtype: float64

It is slightly unbalanced!

In [4]:
import re

user_regex = re.compile(r"@[a-zA-Z0-9_]{0,15}")
url_regex = re.compile(
    "((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|co|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"
)

def preprocess_tweet(text):
    """
    Basic preprocessing
    """
    text = user_regex.sub("usuario", text)
    text = url_regex.sub("url", text)

    return text

In [5]:
train_df["text"] = train_df["text"].apply(preprocess_tweet)
dev_df["text"] = dev_df["text"].apply(preprocess_tweet)
test_df["text"] = test_df["text"].apply(preprocess_tweet)


In [6]:
import torch
from torch.nn import BCEWithLogitsLoss
from torch.nn.functional import binary_cross_entropy_with_logits


"""
Supongamos que tenemos un batch de 32 
Por cada uno
"""

logits = torch.randn(32, 8)
labels = torch.Tensor([[1, 1, 1, 1, 0, 0, 0, 0] for _ in range(32)])


loss_fct = BCEWithLogitsLoss()
loss_fct(logits, labels)


tensor(0.8366)

In [7]:
logits = torch.Tensor([[-10, -9, -10]])
target = torch.zeros(1, 3)

loss_fct(
    logits,
    target,
)

tensor(7.1403e-05)

¿Está haciendo lo esperado esto? Veamos...

Cross entropy es 

$- [y \log \hat{y} + (1-y) \log (1-\hat{y}) ]$

In [8]:
from torch.nn.functional import sigmoid

pred = sigmoid(logits)

losses = -(target * torch.log(pred) + (1 - target) * torch.log(1-pred))

losses.mean()

/home/jmperez/.local/share/virtualenvs/hatespeech-classification-n4GdOxTz/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor(7.1410e-05)

Espectacular!!! 

Qué pasa con el weight?

In [9]:
from torch.nn.functional import sigmoid

pred = sigmoid(logits)

weights = torch.Tensor([0.5, 0.1, 0.4])

losses = -(target * torch.log(pred) + (1 - target) * torch.log(1-pred))

loss_fct = BCEWithLogitsLoss(pos_weight=weights)

(losses * weights).sum(), loss_fct(logits, target)

(tensor(5.3217e-05), tensor(7.1526e-05))



Hummm...no me queda claro **CHEQUEAR ESTO**

In [10]:
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss
from transformers import BertPreTrainedModel, BertTokenizer, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput

class BertForSequenceMultiClassification(BertPreTrainedModel):
    """
    Slight modification of BertForSequenceClassification to allow for multipĺe classification
    heads
    
    In fact, the only modification is the change of the loss! We use a BCEWith
    """
    
    def __init__(self, config):
        """
        The same as BertForSequenceClassification
        """
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            """
            The only thing I change is here!
            """
            loss_fct = BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [11]:


model_name = 'dccuchile/bert-base-spanish-wwm-cased'

device = "cuda" if torch.cuda.is_available() else "cpu"

id2label = {0: 'Not hateful', 1: 'Hateful'}
label2id = {v:k for k,v in id2label.items()}

model = BertForSequenceMultiClassification.from_pretrained(model_name, return_dict=True, num_labels=len(categories))

model.config.hidden_dropout_prob = 0.20
model.config.id2label = id2label
model.config.label2id = label2id

model = model.to(device)
model.train();



tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceMultiClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceMultiClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceMultiClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceMultiClassification were not initialized from the model checkpoint at dccuchile/bert-base

In [12]:
from datasets import Dataset, Value, ClassLabel, Features

#examples = pd.concat([train_df, dev_df])

features = Features({
    'text': Value('string'),
})

for cat in categories:
    features[cat] = ClassLabel(num_classes=2, names=["NO", "YES"])

columns = ["text"] + categories
train_dataset = Dataset.from_pandas(train_df[columns], features=features)
dev_dataset = Dataset.from_pandas(dev_df[columns], features=features)
test_dataset = Dataset.from_pandas(test_df[columns], features=features)


In [13]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

batch_size = 32
eval_batch_size = 16

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
dev_dataset = dev_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)



In [14]:
def format_dataset(dataset):
    def sarasa(examples):
        return {'labels': torch.Tensor([examples[cat] for cat in categories])}
    dataset = dataset.map(sarasa)
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)

Esta API de mierda vive cambiando todo el tiempo

In [15]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    """
    Compute metrics for Trainer
    """
   
    
    
    labels = pred.label_ids
    preds = torch.sigmoid(torch.Tensor(pred.predictions)).round()

    ret = {
    }
    """
    Calculo F1 por cada posición. Asumo que cada categoría está alineada correctamente en la i-ésima posición
    """
    f1s = []
    for i, cat in enumerate(categories):
        cat_labels, cat_preds = labels[:, i], preds[:, i]
        precision, recall, f1, _ = precision_recall_fscore_support(cat_labels, cat_preds, average='macro')
        
        f1s.append(f1)
        
        ret[cat+" F1"] = f1
        
    ret["Mean F1"] = torch.Tensor(f1s).mean()
    return ret

In [16]:
from transformers import Trainer, TrainingArguments
epochs = 3

total_steps = (epochs * len(train_dataset)) // batch_size
warmup_steps = total_steps // 10
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    warmup_steps=warmup_steps,
    evaluation_strategy="epoch",
    do_eval=False,
    weight_decay=0.01,
    logging_dir='./logs',
)

results = []

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Calls f1,Women f1,Lgbti f1,Racism f1,Class f1,Politics f1,Disabled f1,Appearance f1,Criminal f1,Mean f1,Runtime,Samples Per Second
1,No log,0.266307,0.809562,0.720145,0.491102,0.880212,0.477004,0.681807,0.481855,0.824595,0.820352,0.687404,5.328400,241.161000
2,No log,0.208873,0.876902,0.751318,0.764767,0.910400,0.660053,0.824378,0.787424,0.883161,0.855090,0.812610,5.301300,242.394000
3,No log,0.196946,0.876672,0.789809,0.782118,0.911128,0.745986,0.846919,0.811141,0.888411,0.865458,0.835293,5.265300,244.049000


/home/jmperez/.local/share/virtualenvs/hatespeech-classification-n4GdOxTz/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=483, training_loss=0.24680429462567127, metrics={'train_runtime': 247.5683, 'train_samples_per_second': 1.951, 'total_flos': 1300997599810560.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 54763, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 785827, 'train_mem_gpu_alloc_delta': 1381010944, 'train_mem_cpu_peaked_delta': 631922, 'train_mem_gpu_peaked_delta': 3558643712})

In [39]:
def predict(sentence, context=None):
    sentence = preprocess_tweet(sentence)
    idx = torch.LongTensor(tokenizer.encode(sentence)).view(1, -1).to(device)
    output = model(idx)

    output = output.logits.detach().cpu().numpy()[0]

    ret = [cat for cat, out in list(zip(categories, output > 0)) if out]

    return ret

In [40]:
predict("Negro chorro de mierda")

['RACISM']

In [41]:
predict("Quiere bijaaa")

['WOMEN']

In [42]:
predict("Le falta verga")

['LGBTI']

In [43]:
predict("Trolazo")

['LGBTI']

In [44]:
predict("Te llamás Raúl querido")

['LGBTI']

In [45]:
predict("No seas trolo viejo")

['LGBTI', 'APPEARANCE']

In [51]:
predict("A esta trola hay que agarrarla de a varios para que sepa lo que es bueno")

['WOMEN']

Acá no encuentra el llamado a la acción...ahora, si lo cambiamos ligeramente...

In [52]:
predict("A esta negra de mierda hay que agarrarla de a varios para que sepa lo que es bueno")

['calls', 'RACISM']

In [65]:
predict("Hay que encerrar a todos estos homosexuales de mierda")

['calls']

In [66]:
predict("Qué asco de homosexuales")

['LGBTI']

In [67]:
predict("Qué asco de homosexuales, mátenlos")

['calls', 'CRIMINAL']

Parece que correlaciona todo llamado a la acción a los criminales

In [69]:
predict("Negros chorros de mierda, cuarenta ladrillos y arreglamos esto")

['calls', 'RACISM']

In [72]:
predict("negras al paredón!")

['RACISM']

In [74]:
predict("No queremos rojitos acá")

['CLASS', 'POLITICS']

In [76]:
predict("No les gusta laburar eh")

['CLASS']

In [77]:
predict("Agarrá la pala!")

[]

In [78]:
predict("Agarrá la pala, vago!")

['CLASS']

In [79]:
predict("Menos ganas de laborar eh")

['CLASS']

In [80]:
predict("Viven del Estado, esta gente sinceramente no tiene remedio")

['CLASS']